In [71]:
import avro.schema
from avro.io import DatumWriter, DatumReader
from avro.datafile import DataFileWriter, DataFileReader
import tempfile
import numpy as np
from main import LZW
import struct

In [72]:
def pack(data):
    buffer = bytearray([])
    for el in data:
        buffer.extend(struct.pack("H", el+1))
    return buffer

def unpack(buffer: bytearray):
    i=0
    data = []
    while i < len(buffer):
        data.append(struct.unpack("H", buffer[i: i+2])[0]-1)
        i+=2
    return data

In [73]:
lzw = LZW()

with open("test.txt") as file:
    text = file.read()

In [79]:
enc = lzw.encode(text)
bin_data = pack(enc)

with open('test.bn', "wb") as file:
    file.write(bin_data)

In [ ]:
unbin_data = unpack(bin_data)

In [3]:
lzw = LZW()
enc = lzw.encode(text)
schema = avro.schema.parse(open("schema.avsc", "rb").read())
data = {"sonnets": enc}
file = 'compressed_text.avro'

writer = DataFileWriter(open(file, "wb"), DatumWriter(), schema)
writer.append(data)
writer.close()

In [ ]:
reader = DataFileReader(open(file, "rb"), DatumReader())
for t in reader:
    test = t['sonnets']
    dec = lzw.decode(test)
    print(dec)
    
reader.close()

In [1]:
!pip install lzw3


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
!python -m lzw3.compressor -k -t -v -d test.txt

[00:44:18] {MAIN} Executing LZW task with following options:
	recursive:  False
	verbose:    True
	time:       True
	keep:       True
	force:      False
	debug:      True
	files:      ['test.txt']
[00:44:18] {COMPRESSOR} Initializing sequence table; alphabet size = 256
[00:44:18] {COMPRESSOR} Compressing file 'test.txt' to 'test.txt.Z'
'test.txt' compressed from 3.1K to 1.8K - space saved = 41.8% (0ms)
